In [2]:
# ============================================ [ setting ] =======================================================
import numpy as np
import pandas as pd

train = pd.read_csv("/Users/gimjiseong/Downloads/[ DACON ] Game_Behavior_DataAnalysis_Comp./data/train.csv")

In [3]:
## + [ pre-processing ] ======================
df_train = pd.DataFrame(columns = ['game_id']) # game id 
df_train.game_id = train.game_id.unique()

df_train['time'] = np.array(train[train.shift(-1).game_id != train.game_id].time) # game time 

df = train[train.player == 0]
df_train['player0_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species)
df = train[train.player==1]
df_train['player1_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species)

df = train[train.player == 0]
df_train['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
df = train[train.player == 1]
df_train['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()])

for event in train.event.unique():
    df = train[(train.player == 0)&(train.event == event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_train = pd.merge(df_train, df, on='game_id', how='left')

    df = train[(train.player == 1)&(train.event == event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_train = pd.merge(df_train, df, on = 'game_id', how = 'left')
df_train = df_train.fillna(0)

for event in train.event.unique():
    df_train[event + '_diff'] = df_train[event+'_count_0'] - df_train[event+'_count_1']

df_train['winner'] = np.array(train[train.shift(-1).game_id != train.game_id].winner)

In [4]:
idx = df_train[df_train.player0_species=='T'].index
df_train.loc[idx, 'player0_species'] = 0
idx = df_train[df_train.player0_species=='P'].index
df_train.loc[idx, 'player0_species'] = 1
idx = df_train[df_train.player0_species=='Z'].index
df_train.loc[idx, 'player0_species'] = 2

idx = df_train[df_train.player1_species=='T'].index
df_train.loc[idx, 'player1_species'] = 0
idx = df_train[df_train.player1_species=='P'].index
df_train.loc[idx, 'player1_species'] = 1
idx = df_train[df_train.player1_species=='Z'].index
df_train.loc[idx, 'player1_species'] = 2

In [12]:
df_train.head()

,game_id,time,player0_species,player1_species,event_count_0,event_count_1,Camera_count_0,Camera_count_1,Selection_count_0,Selection_count_1,...,ControlGroup_count_1,Camera_diff,Selection_diff,Ability_diff,Right Click_diff,SetControlGroup_diff,GetControlGroup_diff,AddToControlGroup_diff,ControlGroup_diff,winner
0,0,7.24,0,0,592,548,444.0,425.0,50.0,57.0,...,0.0,19.0,-7.0,0.0,7.0,2.0,21.0,2.0,0.0,1
1,1,9.59,1,0,1223,1211,627.0,858.0,186.0,116.0,...,0.0,-231.0,70.0,10.0,29.0,2.0,131.0,1.0,0.0,1
2,2,9.59,1,2,851,1371,413.0,725.0,90.0,232.0,...,2.0,-312.0,-142.0,-16.0,-44.0,5.0,-10.0,1.0,-2.0,0
3,3,9.59,0,1,1389,1622,713.0,388.0,180.0,148.0,...,0.0,325.0,32.0,-7.0,8.0,-13.0,-578.0,0.0,0.0,0
4,4,5.43,0,2,966,651,430.0,272.0,67.0,126.0,...,0.0,158.0,-59.0,21.0,71.0,2.0,125.0,-3.0,0.0,0


In [16]:
# ============================================ [ modeling ] =======================================================
from keras import models 
from keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

x_train = df_train.iloc[:, 1:30]
y_train = df_train["winner"]

Using TensorFlow backend.


In [15]:
model = models.Sequential()
model.add(layers.Dense(units = 4, activation = "relu", input_shape = (x_train.shape[1], )))
model.add(layers.Dense(units = 4, activation = "relu"))
model.add(layers.Dense(units = 1, activation = "sigmoid"))
model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = ["acc"])

NameError: name 'models' is not defined

In [14]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
model.fit(x_train, 
          y_train,
          epochs = 4,
          batch_size = 32)

Epoch 1/4


/bin/sh: conda: command not found
